# Homework #3

## Support Vector Machine Implementation for HOMEWORK 3

In [2]:
import numpy as np
import math

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.svm import SVC

def svmPredict(data, label, svmModel):
    
    predictedLabel = svmModel.predict(data)
    error = 0.0
    
    for i in range(len(label)):
        if predictedLabel[i] < label[i] or predictedLabel[i] > label[i]:
            error += 1.0
    
    error /= float(len(label))
    
    return error

def generateYTest(data, svmModel):
    
    predictedLabel = svmModel.predict(data)
    
    return predictedLabel


#Load in data
given_trainX = np.load("X_train.npy")
given_trainY = np.load("y_train.npy")
given_testX = np.load("X_test.npy")
given_heldoutX = np.load("lfw.heldout.npy")

#to get row and column
height, width = given_trainX.shape

#trainX, testX, trainY, testY = train_test_split(given_trainX, given_trainY, test_size=0.25, random_state=42)

#PCA Algorithm
n_components = 150

pca = PCA(n_components = n_components, svd_solver='randomized', whiten=True).fit(given_trainX)

trainXPCA = pca.transform(given_trainX)
#testXPCA = pca.transform(given_testX)
testXPCA = pca.transform(given_heldoutX)

cList = []
gammaList = []

    
#Fitting the classifier to the training set
param_grid = {'C': [2e-9, 2e-7, 2e-5, 2e-3, 2e-1, 2e1, 2e3, 2e5, 2e7],
              'gamma': [2e-9, 2e-7, 2e-5, 2e-3, 2e-1, 2e1, 2e3, 2e5, 2e7],}

classifier = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
classifier = classifier.fit(trainXPCA, given_trainY)

predictedTestY = generateYTest(testXPCA, classifier)

data = np.empty((len(predictedTestY), 2), dtype='int')
for i in range (len(predictedTestY)):
        data[i][0] = i
        data[i][1] = predictedTestY[i]

np.savetxt('output.txt', data, delimiter=',', fmt='%d', header="ImageId,PredictedClass")